In [1]:
# pip install
!pip install datasets==2.18.0 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==1

In [2]:
# import library
import torch
import torch .nn as nn

from PIL import Image
from datasets import load_dataset
from torch.utils.data import Dataset , DataLoader
from torchvision.models import resnet18
from torchvision import transforms

In [3]:
# Load dataset
DATASET_NAME = 'cats_vs_dogs'
datasets = load_dataset (DATASET_NAME)
datasets

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/23410 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 23410
    })
})

In [4]:
# Split the train/val dataset
TEST_SIZE = 0.2
datasets = datasets ['train']. train_test_split (test_size = TEST_SIZE)

In [5]:
# Build dataloader
IMG_SIZE = 64
img_transforms = transforms.Compose ([
transforms.Resize (( IMG_SIZE , IMG_SIZE ) ) ,
transforms.Grayscale ( num_output_channels =3) ,
transforms.ToTensor () ,
transforms.Normalize (
[0.485 , 0.456 , 0.406] ,
[0.229 , 0.224 , 0.225]
)
])

In [6]:
# Build class CatDogDataset
class CatDogDataset ( Dataset ) :
  def __init__ (self , data , transform = None ) :
    self.data = data
    self.transform = transform

  def __len__ ( self ) :
    return len( self.data )

  def __getitem__ (self , idx) :
    images = self.data [ idx ]['image']
    labels = self.data [ idx ]['labels']

    if self.transform :
      images = self.transform ( images )
    labels = torch.tensor ( labels , dtype = torch.long )
    return images , labels

In [7]:
# Declare DataLoader
TRAIN_BATCH_SIZE = 512
VAL_BATCH_SIZE = 256

train_dataset = CatDogDataset ( datasets ['train'] , transform = img_transforms )
test_dataset = CatDogDataset ( datasets ['test'] , transform = img_transforms )

train_loader = DataLoader ( train_dataset , batch_size = TRAIN_BATCH_SIZE ,
shuffle = True )
test_loader = DataLoader ( test_dataset , batch_size = VAL_BATCH_SIZE , shuffle =
False )

In [8]:
# Build Model
class CatDogModel (nn. Module ) :
  def __init__ (self , n_classes ) :
    super ( CatDogModel , self ).__init__ ()

    resnet_model = resnet18 ( weights ='IMAGENET1K_V1')
    self.backbone = nn. Sequential (* list ( resnet_model.children () ) [: -1])
    for param in self.backbone.parameters () :
      param.requires_grad = False

    in_features = resnet_model .fc. in_features
    self .fc = nn. Linear ( in_features , n_classes )

  def forward (self , x) :
    x = self.backbone (x)
    x = torch.flatten (x, 1)
    x = self .fc(x)

    return x

In [9]:
device = 'cuda' if torch.cuda.is_available () else 'cpu'
N_CLASSES = 2
model = CatDogModel ( N_CLASSES ) .to( device )
test_input = torch.rand (1 , 3 , 224 , 224) .to( device )
with torch.no_grad () :
  output = model ( test_input )
  print ( output.shape ) # (1 , 2)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 153MB/s]


torch.Size([1, 2])


In [10]:
# Training model
EPOCHS = 20
LR = 1e-3
WEIGHT_DECAY = 1e-5

optimizer = torch.optim.Adam ( model.parameters () , lr=LR , weight_decay = WEIGHT_DECAY )
criterion = torch .nn. CrossEntropyLoss ()

for epoch in range ( EPOCHS ) :
  train_losses = []
  model.train ()
  for images , labels in train_loader :
    images = images .to( device )
    labels = labels .to( device )

    outputs = model ( images )

    optimizer.zero_grad ()
    loss = criterion ( outputs , labels )
    loss.backward ()
    optimizer.step ()

    train_losses.append ( loss.item () )

  train_loss = sum( train_losses ) / len( train_losses )

  val_losses = []
  model.eval ()
  with torch.no_grad () :
    for images , labels in test_loader :
      images = images .to( device )
      labels = labels .to( device )

      outputs = model ( images )
      loss = criterion ( outputs , labels )

      val_losses.append ( loss.item () )

  val_loss = sum( val_losses ) / len( val_losses )

  print (f'EPOCH { epoch + 1}:\ tTrain loss : { train_loss:.3f}\ tVal loss : {val_loss:.3f}')

EPOCH 1:\ tTrain loss : 0.640\ tVal loss : 0.597
EPOCH 2:\ tTrain loss : 0.543\ tVal loss : 0.544
EPOCH 3:\ tTrain loss : 0.523\ tVal loss : 0.540
EPOCH 4:\ tTrain loss : 0.514\ tVal loss : 0.534
EPOCH 5:\ tTrain loss : 0.509\ tVal loss : 0.535
EPOCH 6:\ tTrain loss : 0.506\ tVal loss : 0.532
EPOCH 7:\ tTrain loss : 0.504\ tVal loss : 0.531
EPOCH 8:\ tTrain loss : 0.501\ tVal loss : 0.534
EPOCH 9:\ tTrain loss : 0.503\ tVal loss : 0.530
EPOCH 10:\ tTrain loss : 0.500\ tVal loss : 0.530
EPOCH 11:\ tTrain loss : 0.500\ tVal loss : 0.529
EPOCH 12:\ tTrain loss : 0.498\ tVal loss : 0.537
EPOCH 13:\ tTrain loss : 0.499\ tVal loss : 0.532
EPOCH 14:\ tTrain loss : 0.500\ tVal loss : 0.529
EPOCH 15:\ tTrain loss : 0.498\ tVal loss : 0.535
EPOCH 16:\ tTrain loss : 0.498\ tVal loss : 0.532
EPOCH 17:\ tTrain loss : 0.498\ tVal loss : 0.530
EPOCH 18:\ tTrain loss : 0.495\ tVal loss : 0.529
EPOCH 19:\ tTrain loss : 0.497\ tVal loss : 0.529
EPOCH 20:\ tTrain loss : 0.499\ tVal loss : 0.536


In [11]:
# Save model weight
SAVE_PATH = 'catdog_weights.pt'
torch.save ( model.state_dict () , SAVE_PATH )